In [ ]:
import numpy as np
import training_data_generation as tdg
import matplotlib.pyplot as plt
%matplotlib inline
num_examples = 10
run_dir = "Data"

# Generate and Save Training Data

In [ ]:
dataset_params, sys_params, facility_spec = tdg.main((_, run_dir, num_examples))

# Diagnostic

In [ ]:
import healpy as hp
import utils_intensity_map as uim
import utils_deck_generation as idg
import netcdf_read_write as nrw
import training_data_generation as tdg
# sys_params["run_clean"] must be false before run for diagnostic

In [ ]:
diag_dir = "Data"

sys_params = tdg.define_system_params(diag_dir)
sys_params["root_dir"] = diag_dir
sys_params["run_gen_deck"] = False
dataset_params, facility_spec = tdg.define_dataset_params(int(num_examples))
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)
iex = 0

run_location = sys_params["root_dir"] + "/" + sys_params["sim_dir"] + str(iex)
theta_slice = slice(0,29,4)
phi_slice   = slice(1,30,4)
power_slice = slice(3,32,4)

cone_theta_offset = dataset_params["sim_params"][theta_slice,iex]
cone_phi_offset = dataset_params["sim_params"][phi_slice,iex]
cone_powers = dataset_params["sim_params"][power_slice,iex]

beams_prev = 0
beams_tot = 0
total_power = 0
for icone in range(facility_spec['num_cones']):
    beams_per_cone = facility_spec['beams_per_cone'][icone]
    beams_tot += beams_per_cone
    total_power += cone_powers[icone] * beams_per_cone
    beams_prev += beams_per_cone
mean_power_fraction = total_power / facility_spec['nbeams']

intensity_map = nrw.read_intensity(run_location, dataset_params["imap_nside"])
intensity_map_rms_spatial = uim.readout_intensity(facility_spec, intensity_map, mean_power_fraction)

hp.mollview(intensity_map,unit=r"$\rm{W/cm^2}$",flip="geo")
hp.graticule()
port_theta = facility_spec["Port_centre_theta"]
port_phi = facility_spec["Port_centre_phi"]
hp.projscatter(port_theta, port_phi)
hp.projscatter(dataset_params["theta_pointings"][:,iex], dataset_params["phi_pointings"][:,iex])
plt.savefig(sys_params["figure_location"]+'/intensity_mollweide_original'+str(iex)+'.png', dpi=300, bbox_inches='tight')

In [ ]:
hp.mollview(intensity_map_rms_spatial, unit="Deviation from mean (%)",flip="geo")
plt.savefig(sys_params["figure_location"]+'/deviation_from_mean_mollweide.png', dpi=300, bbox_inches='tight')
hp.graticule()

In [ ]:
LMAX = dataset_params["LMAX"]
power_spectrum_unweighted, power_spectrum_weighted = uim.power_spectrum(intensity_map, LMAX)
x_max = 20

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_unweighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Unweighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_original"+str(iex)+".png", dpi=300, bbox_inches='tight')

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_weighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Weighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/weighted_modes.png", dpi=300, bbox_inches='tight')

In [ ]:
intensity_map_normalized, avg_power = uim.imap_norm(intensity_map)
hp.mollview(intensity_map_normalized, unit="Deviation from Mean", flip="geo")

X_train = uim.imap2xtrain(intensity_map_normalized, LMAX, avg_power)
intensity_map_normalized2 = uim.xtrain2imap(X_train, LMAX, dataset_params["imap_nside"], avg_power)
hp.mollview(intensity_map_normalized2, unit="Deviation from Mean",flip="geo")

# Combine Datasets

In [ ]:
from netCDF4 import Dataset
from os import path
import os
import numpy as np

In [ ]:
dataset1 = "Data1"
dataset2 = "Data2"

new_file = "Data3"

filename_trainingdata = dataset1 + "/" + sys_params["trainingdata_filename"] 
training_data = Dataset(filename_trainingdata)
X_1 = training_data.variables["X_train"][:]
Y_1 = training_data.variables["Y_train"][:]
avg_powers_1 = training_data.variables["avg_powers"][:]
training_data.close()

filename_trainingdata = dataset2 + "/" + sys_params["trainingdata_filename"] 
training_data = Dataset(filename_trainingdata)
X_2 = training_data.variables["X_train"][:]
Y_2 = training_data.variables["Y_train"][:]
avg_powers_2 = training_data.variables["avg_powers"][:]
training_data.close()

X_train = np.hstack((X_1, X_2))
Y_train = np.hstack((Y_1, Y_2))
avg_powers = np.hstack((avg_powers_1, avg_powers_2))
print(np.shape(X_train), np.shape(Y_train), np.shape(avg_powers))

filename_trainingdata = new_file + "/" + sys_params["trainingdata_filename"] 
nrw.save_training_data(X_train, Y_train, avg_powers, filename_trainingdata)

# Neural Network

In [ ]:
import neural_network_generation as nng
num_epochs = 100
num_nn = 1

In [ ]:
nn_params, nn_dataset, sys_params, nn_hyperparams = nng.main((_, run_dir, num_epochs, num_nn))

In [ ]:
fig = plt.figure()
ax1 = plt.axes()
ax1.plot(nn_hyperparams["initialize_seed"], nn_hyperparams["test_acc"], "x")
ax1.set_ylabel('Absolute Test Accuracy')
ax1.set_xlabel('Neural Network Random Seed')

## Compare NN

In [ ]:
import netcdf_read_write as nrw

In [ ]:
filename_hyperparams = nn_params["dir_nn_weights"] + "/" + nn_params["filename_hyperparams"]
nn_hyperparams = nrw.read_nn_weights(filename_hyperparams)

fig = plt.figure()
ax1 = plt.axes()
ax1.plot(nn_hyperparams["initialize_seed"], nn_hyperparams["test_acc"], "x")
ax1.set_ylabel('Absolute Test Accuracy')
ax1.set_xlabel('Random Seed')

## Compare NN accuracy from weights

In [ ]:
import tf_neural_network as tfnn
import netcdf_read_write as nrw
import neural_network_generation as nng

In [ ]:
num_nn = 1
initialize_seed = np.arange(num_nn)
test_acc = np.zeros(num_nn)

sys_params = tdg.define_system_params(run_dir)
nn_params = nng.define_nn_params(num_nn)
X_all, Y_all, avg_powers_all, nn_params = nng.import_training_data(nn_params, sys_params)
nn_params["test_fraction"] = 0.0
nn_params["test_size"] = 0
nn_dataset = nng.seperate_test_set(X_all, Y_all, avg_powers_all, nn_params)
nn_dataset = nng.normalise(nn_dataset)

for inn in range(num_nn):
    nn_weights = nrw.read_nn_weights(nn_params["dir_nn_weights"] + "/" + "NN" + str(inn))
    y_pred = tfnn.apply_network(nn_dataset["X_train"], nn_weights)
    test_acc[inn] = np.mean(np.abs(nn_dataset["Y_train"] - y_pred.T))

fig = plt.figure()
ax1 = plt.axes()
ax1.plot(initialize_seed, test_acc, "x")
ax1.set_ylabel('Absolute Test Accuracy')
ax1.set_xlabel('Neural Network Random Seed')
plt.savefig(sys_params["figure_location"]+'/Compare_NN.png', dpi=300, bbox_inches='tight')

# Run Prediction Through Ifriit

In [ ]:
import matplotlib.pyplot as plt
import healpy as hp
import tf_neural_network as tfnn
import netcdf_read_write as nrw
import utils_deck_generation as idg
import training_data_generation as tdg
import utils_intensity_map as uim
import neural_network_generation as nng
%matplotlib inline
num_epochs = 10
num_nn = -1

In [ ]:
inn = 0
num_nn = 1
nn_params = nng.define_nn_params(num_nn)
test_pred_dir = "Data_test_pred_1"

# choose neural network
nn_weights = nrw.read_nn_weights(nn_params["dir_nn_weights"] + "/" + "NN" + str(inn))

# choose test data set
sys_params = tdg.define_system_params("Data")
nn_params = nng.define_nn_params(num_nn)
X_all, Y_all, avg_powers_all, nn_params = nng.import_training_data(nn_params, sys_params)
nn_params["test_fraction"] = 0.0
nn_params["test_size"] = 0
nn_dataset = nng.seperate_test_set(X_all, Y_all, avg_powers_all, nn_params)
nn_dataset = nng.normalise(nn_dataset)
dataset_params, facility_spec = tdg.define_dataset_params(nn_params["num_examples"])
dataset_params["Y_train"] = nn_dataset["Y_train"].T
y_true = dataset_params["Y_train"]

# apply neural net to test set
sys_params = tdg.define_system_params(test_pred_dir)
y_pred = tfnn.apply_network(nn_dataset["X_train"], nn_weights)
dataset_params["Y_train"] = y_pred

print("The absolute error is: ", np.mean(np.abs(y_true - y_pred)))
print("The percentage relative error is: ", np.mean((np.abs(y_true - y_pred)) / y_true) * 100, "%")

In [ ]:
# Create new run files
sys_params["run_clean"] = False
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)
tdg.generate_training_data(dataset_params, sys_params, facility_spec)

In [ ]:
sys_params = tdg.define_system_params(test_pred_dir)
sys_params["run_gen_deck"] = False
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)
# plot comparison
iex = 0

run_location = sys_params["root_dir"] + "/" + sys_params["sim_dir"] + str(iex)
theta_slice = slice(0,29,4)
phi_slice   = slice(1,30,4)
power_slice = slice(3,32,4)

cone_theta_offset = dataset_params["sim_params"][theta_slice,iex]
cone_phi_offset = dataset_params["sim_params"][phi_slice,iex]
cone_powers = dataset_params["sim_params"][power_slice,iex]

beams_prev = 0
beams_tot = 0
total_power = 0
for icone in range(facility_spec['num_cones']):
    beams_per_cone = facility_spec['beams_per_cone'][icone]
    beams_tot += beams_per_cone
    total_power += cone_powers[icone] * beams_per_cone
    beams_prev += beams_per_cone
mean_power_fraction = total_power / facility_spec['nbeams']

intensity_map = nrw.read_intensity(run_location, dataset_params["imap_nside"])
intensity_map_rms_spatial = uim.readout_intensity(facility_spec, intensity_map, mean_power_fraction)

hp.mollview(intensity_map,unit=r"$\rm{W/cm^2}$",flip="geo")
hp.graticule()
port_theta = facility_spec["Port_centre_theta"]
port_phi = facility_spec["Port_centre_phi"]
hp.projscatter(port_theta, port_phi)
hp.projscatter(dataset_params["theta_pointings"][:,iex], dataset_params["phi_pointings"][:,iex])
plt.savefig(sys_params["figure_location"]+'/intensity_mollweide_nn'+ str(iex) +'.png', dpi=300, bbox_inches='tight')

In [ ]:
LMAX = dataset_params["LMAX"]
power_spectrum_unweighted, power_spectrum_weighted = uim.power_spectrum(intensity_map, LMAX)
x_max = 20

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_unweighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Unweighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_nn"+ str(iex) +".png", dpi=300, bbox_inches='tight')

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_weighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Weighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/weighted_modes.png", dpi=300, bbox_inches='tight')

# Run PDD through NN

In [ ]:
import matplotlib.pyplot as plt
import healpy as hp
import tf_neural_network as tfnn
import netcdf_read_write as nrw
import utils_deck_generation as idg
import training_data_generation as tdg
import utils_intensity_map as uim
import neural_network_generation as nng
import utils_intensity_map as uim
%matplotlib inline
num_epochs = 10
num_examples = 1

In [ ]:
imap_nside = 256
LMAX = 30
input_size = 992
output_size = 16
nif_pdd_dir = "Data_pdd"

run_location = nif_pdd_dir + "/run_0"
intensity_map = nrw.read_intensity(run_location, imap_nside)
X_train, avg_powers_all = uim.create_xtrain(intensity_map, LMAX)
X_all = np.reshape(X_train, (-1,input_size))
Y_all = np.zeros(output_size)

# choose neural network
inn = 0
num_nn = 1
nn_weights = nrw.read_nn_weights("neural_network_weights" + "/" + "NN" + str(inn))

# param setup
nif_pdd_pred_dir = "Data_pdd_pred"
sys_params = tdg.define_system_params(nif_pdd_pred_dir)
nn_params = nng.define_nn_params(num_nn)
nn_params["test_fraction"] = 0.0
nn_params["test_size"] = 0
nn_dataset = nng.seperate_test_set(X_all, Y_all, avg_powers_all, nn_params)
nn_dataset = nng.normalise(nn_dataset)

y_pred = tfnn.apply_network(nn_dataset["X_train"].T, nn_weights)
print(y_pred)

dataset_params, facility_spec = tdg.define_dataset_params(num_examples)
dataset_params["Y_train"] = y_pred

In [ ]:
# Create new run files
sys_params["run_clean"] = False
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)
tdg.generate_training_data(dataset_params, sys_params, facility_spec)

In [ ]:
sys_params = tdg.define_system_params(nif_pdd_pred_dir)
sys_params["run_gen_deck"] = False
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)
# plot comparison
iex = 0

run_location = sys_params["root_dir"] + "/" + sys_params["sim_dir"] + str(iex)
theta_slice = slice(0,29,4)
phi_slice   = slice(1,30,4)
power_slice = slice(3,32,4)

cone_theta_offset = dataset_params["sim_params"][theta_slice,iex]
cone_phi_offset = dataset_params["sim_params"][phi_slice,iex]
cone_powers = dataset_params["sim_params"][power_slice,iex]

beams_prev = 0
beams_tot = 0
total_power = 0
for icone in range(facility_spec['num_cones']):
    beams_per_cone = facility_spec['beams_per_cone'][icone]
    beams_tot += beams_per_cone
    total_power += cone_powers[icone] * beams_per_cone
    beams_prev += beams_per_cone
mean_power_fraction = total_power / facility_spec['nbeams']

intensity_map = nrw.read_intensity(run_location, dataset_params["imap_nside"])
intensity_map_rms_spatial = uim.readout_intensity(facility_spec, intensity_map, mean_power_fraction)

hp.mollview(intensity_map,unit=r"$\rm{W/cm^2}$",flip="geo")
hp.graticule()
port_theta = facility_spec["Port_centre_theta"]
port_phi = facility_spec["Port_centre_phi"]
hp.projscatter(port_theta, port_phi)
hp.projscatter(dataset_params["theta_pointings"][:,iex], dataset_params["phi_pointings"][:,iex])
plt.savefig(sys_params["figure_location"]+'/intensity_mollweide_pred.png', dpi=300, bbox_inches='tight')

In [ ]:
LMAX = dataset_params["LMAX"]
power_spectrum_unweighted, power_spectrum_weighted = uim.power_spectrum(intensity_map, LMAX)
x_max = 20

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_unweighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Unweighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_pred.png", dpi=300, bbox_inches='tight')

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_weighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Weighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/weighted_modes.png", dpi=300, bbox_inches='tight')